In [1]:
import chess

import pandas as pd
import matplotlib.pyplot as plt
import encoding_tools as EncodingTools

from model import ChessNet

import torch
from torch.utils.data import DataLoader, Dataset


MODE = "DEBUG"  # If in release mode, please comment this line
# MODE = "RELEASE"

In [2]:
from tqdm import tqdm
from FEN_to_chessboard import FenToChessBoard
from encoder_decoder import *

# Pulling in training data using Pandas
df = pd.concat([
    pd.read_csv('stockfish_data/chess_games_1.csv'),
    pd.read_csv('stockfish_data/chess_games_2.csv'),
    pd.read_csv('stockfish_data/chess_games_2.csv')]
)

non_zero_winners = df[df['Winner'] != 0].copy()
non_zero_winners.reset_index(drop=True, inplace=True)
print("Game with an existed winner:", non_zero_winners.shape) # (18830, 4)

train_df = non_zero_winners[:5000] if MODE == "DEBUG" else non_zero_winners[:16000]
# We'll also grab the last 1000 examples as a validation set
val_df = non_zero_winners[-1000:] if MODE == "DEBUG" else non_zero_winners[-2800:]

##### Package the training data

X_train = np.stack(train_df['FEN'].apply(FenToChessBoard.fen_to_board).apply(encode_board)).reshape(-1, 22, 8, 8) # Size(5000, 22, 8, 8)
print("Training set size:", X_train.shape)

train_best_move_embedding = []
for idx, row in tqdm(train_df.iterrows(), total=len(train_df), desc="[TrainSet] BestMove to embedding"):
    fen_str = row['FEN']
    move_str = row['BestMove']
    # Check if fen_str is a valid string before processing
    if not isinstance(fen_str, str):
        raise ValueError(f"Invalid FEN string at index {idx}: {fen_str}")
    board = FenToChessBoard.fen_to_board(fen_str)
    train_best_move_embedding.append(move_on_board(board, move_str))
train_best_move_embedding = np.array(train_best_move_embedding)
print("Embedded Best move shape:", train_best_move_embedding.shape)
y_train = {'best_move' : train_best_move_embedding, 'winner' : train_df['Winner']}


##### Package the validation data

X_val = np.stack(val_df['FEN'].apply(FenToChessBoard.fen_to_board).apply(encode_board)).reshape(-1, 22, 8, 8)
print("Validation set size:", X_val.shape)

val_best_move_embedding = []
for idx, row in tqdm(val_df.iterrows(), total=len(val_df), desc="[ValSet] BestMove to embedding"):
    fen_str = row['FEN']
    move_str = row['BestMove']
    # Check if fen_str is a valid string before processing
    if not isinstance(fen_str, str):
        raise ValueError(f"Invalid FEN string at index {idx}: {fen_str}")
    board = FenToChessBoard.fen_to_board(fen_str)
    val_best_move_embedding.append(move_on_board(board, move_str))
val_best_move_embedding = np.array(val_best_move_embedding)
X_val_cur_board = np.stack(val_df['FEN'].apply(FenToChessBoard.fen_to_board))
y_val = {'best_move' : val_best_move_embedding, 'winner' : val_df['Winner']}

Game with an existed winner: (18830, 4)
Training set size: (5000, 22, 8, 8)


[TrainSet] BestMove to embedding: 100%|██████████| 5000/5000 [00:00<00:00, 10658.85it/s]


Embedded Best move shape: (5000, 4672)
Validation set size: (1000, 22, 8, 8)


[ValSet] BestMove to embedding: 100%|██████████| 1000/1000 [00:00<00:00, 5826.39it/s]


In [3]:
# Instantiate the model
model = ChessNet()

# Move tensors to device if CUDA or MPS is available
if torch.cuda.is_available():
    device = "cuda"
# elif torch.mps.is_available(): # For M series chips of Mac
#     device = "mps"
else:
    device = "cpu"

model = model.to(device)

In [4]:
from model import ChessDataset

# Create Dataset objects for training and validation
train_dataset = ChessDataset(X_train, y_train)
val_dataset = ChessDataset(X_val, y_val)

# Create DataLoaders for batching
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

for X_batch, y_batch in train_loader:
    # 获取一个批次的数据
    best_move_batch, winner_batch = y_batch  # 拆分 y_batch
    print("X_batch shape:", X_batch.shape)  # 打印 X 的维度
    print("best_move shape:", best_move_batch.shape)  # 打印 best_move 的维度
    print("winner shape:", winner_batch.shape)  # 打印 winner 的维度
    break  # 打印一个批次后停止

X_batch shape: torch.Size([256, 22, 8, 8])
best_move shape: torch.Size([256, 4672])
winner shape: torch.Size([256])


In [ ]:
import torch.optim as optim
from train import AlphaLoss

n_epochs = 40
learning_rate = 0.003
train_losses, val_losses = [], []
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = AlphaLoss().to(device)

for epoch in range(n_epochs):
    model.train()  # Set model to training mode
    epoch_train_loss = 0

    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{n_epochs} [Training]"):
        # print("Size of X_batch:", X_batch.shape)
        # print("Size of p in Y_batch:", y_batch[0].shape)
        # print("Size of v in Y_batch:", y_batch[1].shape)
        X_batch = X_batch.to(device)
        y_p = y_batch[0].to(device)
        y_v = y_batch[1].to(device).reshape(-1, 1)

        optimizer.zero_grad()
        predictions = model(X_batch)
        # print("Size of p in predictions:", predictions['p'].shape)
        # print("Size of v in predictions:", predictions['v'].shape)
        loss = loss_fn(y_v, predictions['v'], y_p, predictions['p'])
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()

    # Average training loss for the epoch
    train_losses.append(epoch_train_loss / len(train_loader))
    print(f"Epoch {epoch + 1}: Training Loss = {train_losses[-1]}")

    # Validation loop
    model.eval()  # Set model to evaluation mode
    epoch_val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{n_epochs} [Validation]"):
            predictions = model(X_batch)
            loss = loss_fn(predictions, y_batch)
            epoch_val_loss += loss.item()

    # Average validation loss for the epoch
    val_losses.append(epoch_val_loss / len(val_loader))
    print(f"Epoch {epoch + 1}: Validation Loss = {val_losses[-1]}")

Epoch 1/40 [Training]:   0%|          | 0/20 [00:00<?, ?it/s]

Size of X_batch: torch.Size([256, 22, 8, 8])
Size of p in Y_batch: torch.Size([256, 4672])
Size of v in Y_batch: torch.Size([256])
Size of input to ConvBlock: torch.Size([256, 22, 8, 8])
Size of p in predictions: torch.Size([256, 4672])
Size of v in predictions: torch.Size([256, 1])


Epoch 1/40 [Training]:   5%|▌         | 1/20 [00:06<02:03,  6.49s/it]

Size of X_batch: torch.Size([256, 22, 8, 8])
Size of p in Y_batch: torch.Size([256, 4672])
Size of v in Y_batch: torch.Size([256])
Size of input to ConvBlock: torch.Size([256, 22, 8, 8])
Size of p in predictions: torch.Size([256, 4672])
Size of v in predictions: torch.Size([256, 1])


Epoch 1/40 [Training]:  10%|█         | 2/20 [00:13<02:03,  6.87s/it]

Size of X_batch: torch.Size([256, 22, 8, 8])
Size of p in Y_batch: torch.Size([256, 4672])
Size of v in Y_batch: torch.Size([256])
Size of input to ConvBlock: torch.Size([256, 22, 8, 8])
Size of p in predictions: torch.Size([256, 4672])
Size of v in predictions: torch.Size([256, 1])


Epoch 1/40 [Training]:  15%|█▌        | 3/20 [00:22<02:11,  7.76s/it]

Size of X_batch: torch.Size([256, 22, 8, 8])
Size of p in Y_batch: torch.Size([256, 4672])
Size of v in Y_batch: torch.Size([256])
Size of input to ConvBlock: torch.Size([256, 22, 8, 8])
Size of p in predictions: torch.Size([256, 4672])
Size of v in predictions: torch.Size([256, 1])


Epoch 1/40 [Training]:  20%|██        | 4/20 [00:32<02:17,  8.58s/it]

Size of X_batch: torch.Size([256, 22, 8, 8])
Size of p in Y_batch: torch.Size([256, 4672])
Size of v in Y_batch: torch.Size([256])
Size of input to ConvBlock: torch.Size([256, 22, 8, 8])
Size of p in predictions: torch.Size([256, 4672])
Size of v in predictions: torch.Size([256, 1])


In [ ]:
# Plotting results (optional)
plt.style.use('ggplot')
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.legend()
plt.title('Loss During Training')
plt.show()

In [ ]:
# Implementing our model as a function
def play_nn(fen, show_move_evaluations=False):
    # We can create a python-chess board instance from the FEN string like this:
    board = chess.Board(fen=fen)

    # And then evaluate all legal moves
    moves = []
    input_vectors = []
    for move in board.legal_moves:
        # For each move, we'll make a copy of the board and try that move out
        candidate_board = board.copy()
        candidate_board.push(move)
        moves.append(move)
        input_vectors.append(EncodingTools.encode_board(str(candidate_board)).astype(np.int32).flatten())

    input_vectors = np.stack(input_vectors)
    # This is where our model gets to shine! It tells us how good the resultant score board is for black:
    scores = model.predict(input_vectors, verbose=0)
    # argmax gives us the index of the highest scoring move
    if board.turn == chess.BLACK:
        index_of_best_move = np.argmax(scores)
    else:
        # If we're playing as white, we want black's score to be as small as possible, so we take argmax of the negative of our array
        index_of_best_move = np.argmax(-scores)

    if show_move_evaluations:
        print(zip(moves, scores))

    best_move = moves[index_of_best_move]

    # Now we turn our move into a string, return it and call it a day!
    return str(best_move)

In [ ]:
# Now we'll import our test set, and make some final predictions!

test_df = pd.read_csv('datasets/test.csv')

test_df.head()

In [ ]:
# Making all of our predictions happens in this one line!
# We're basically saying "run play_nn on all the boards in the test_df, and then keep the results as best_move"
# Because this invovles running our model a _ton_ this step will take a while.

test_df['best_move'] = test_df['board'].apply(play_nn)

In [ ]:
test_df['best_move']

In [ ]:
# Let's make sure our submission looks like the sample submission
submission = test_df[['id', 'best_move']]
print(submission.head())

sample_submission = pd.read_csv('datasets/sample_submission.csv', index_col='id')
print(sample_submission.head())

In [ ]:
# We should not output the submission file
# submission.to_csv('submission.csv', index=False)